### Time Estimate


In [ ]:
def to_minutes(x):
    hours, minutes = map(int, x.split(":"))
    return hours*60+minutes

PRETRAINING = to_minutes("3:16")
INFLUENCE = to_minutes("6:00")
EVALUATION = to_minutes("0:30")



In [2]:
total = 0

In [3]:
import config
from itertools import product


In [4]:
pretraining_jobs = list(product(config.datasets, config.model_types, config.baseline_curricula))  +\
     [(dataset, model_type, model_type + curriculum) for dataset, model_type, curriculum in product(config.datasets, config.model_types, config.influence_curricula)]
       
print(len(pretraining_jobs))
total += len(pretraining_jobs) * PRETRAINING

66


In [5]:
influence_estimation_jobs = list(product(config.datasets, config.model_types))
   
print(len(influence_estimation_jobs))
total += len(influence_estimation_jobs) * INFLUENCE

6


In [6]:

total += len(pretraining_jobs) * EVALUATION

In [7]:
def format_minutes(minutes):
    days = minutes // 1440 
    hours = (minutes % 1440) // 60
    mins = minutes % 60
    parts = []
    if days:
        parts.append(f"{days} day{'s' if days > 1 else ''}")
    if hours:
        parts.append(f"{hours} hour{'s' if hours > 1 else ''}")
    if mins or not parts:
        parts.append(f"{mins} minute{'s' if mins > 1 else ''}")
    
    return ", ".join(parts)
print("sequential:",format_minutes(total))
print("8 gpus:",format_minutes(total/2))
print("12 gpus:",format_minutes(total/3))

sequential: 13 days, 1 hour, 12 minutes
8 gpus: 6.0 days, 12.0 hours, 36.0 minutes
12 gpus: 4.0 days, 8.0 hours, 24.0 minutes
